In [36]:
import json
import pickle

import numpy as np
import pandas as pd
from scipy import stats

reg_prediction_files = ['2000', '2000_4000', '4000_6000', '6000_8000', '8000_10000', '10000_11000', '11000_15000', '15000_17000', '17000_20000', '20000_end']
all_preds = []

# Loop through each file and load the predictions
for file in reg_prediction_files:
    with open(f"predictions/y_pred_reg_{file}.pkl", "rb") as f:
        reg_preds = pickle.load(f)
        print(len(reg_preds))
        flattened_tensor_list = [item for sublist in reg_preds for item in sublist]
        preds = []
        for j in range(len(flattened_tensor_list)):
            if isinstance(flattened_tensor_list[j], bool):
                preds.append(flattened_tensor_list[j])
                continue
            
            preds.append(flattened_tensor_list[j].numpy()[0][0])
        print(len(preds))
        all_preds.append(preds)
        print(len(all_preds))



2000
20000
1
2000
20000
2
2000
20000
3
2000
20000
4
2000
20000
5
1000
10000
6
4000
40000
7
2000
20000
8
3000
30000
9
1436
14360
10


In [38]:
combined_reg_preds = np.concatenate(all_preds, axis=0)

In [39]:
len(combined_reg_preds)

214360

In [40]:
# Concatenate all predictions into a single array
combined_reg_preds = np.concatenate(all_preds, axis=0)

# Optionally, save the combined predictions to a new file
with open("predictions/combined_reg_preds.pkl", "wb") as f:
    pickle.dump(combined_reg_preds, f)

with open("predictions/class_predictions.pkl", "rb") as f:
    class_preds = pickle.load(f)

In [48]:
y_preds_10 = np.array(combined_reg_preds).reshape(-1, 10)

# Calculate the Z-scores
z_scores = stats.zscore(
    y_preds_10, axis=1
)  # Calculate Z-scores for the entire array

# Create a boolean mask where elements with |Z-score| > 3 are True
outliers_mask = np.abs(z_scores) > 2.5

# Convert outliers to False
y_preds_10[outliers_mask] = False

print(y_preds_10)

[[ 9.27094227  8.93795492  8.59709037 ...  8.72924028  8.5579202
   0.        ]
 [ 1.82612505  1.62816567  1.94056469 ...  2.50960277  1.6865316
   1.90979697]
 [-3.33576015 -5.03238398 -4.62953495 ... -3.27101134  0.
  -6.32220861]
 ...
 [-3.81363774 -4.65818414  0.26042204 ... -1.60947944 -4.00797423
  -1.24212375]
 [12.55944631 12.15027597 12.16126168 ... 12.8047358  12.84917697
  15.33892423]
 [ 6.11198372  5.95945906  0.         ...  5.76481379  6.12867991
   5.61380804]]


In [49]:
y_preds_10.shape

(21436, 10)

In [50]:
y_preds_10[y_preds_10 == 0] = None

# Calculate the mean of each row, ignoring None values (use np.nanmean)
row_means = np.nanmean(y_preds_10.astype(float), axis=1)
row_stds = np.nanstd(y_preds_10.astype(float), axis=1)


C:\Users\janni\AppData\Local\Temp\ipykernel_19896\1627597677.py:4: RuntimeWarning: Mean of empty slice
  row_means = np.nanmean(y_preds_10.astype(float), axis=1)
c:\Users\janni\anaconda3\envs\NUMP\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [51]:
output_data = {}
for i in range(len(row_means)):

    output_data[i] = {
            #"class": np.int64(class_preds[i]),
            "class_conf": 1,
            "pdf_type": "norm",
            "pdf_args": {"loc": row_means[i], "scale": row_stds[i]},
    }

In [53]:
output_data[0]["class_conf"].dtype

AttributeError: 'int' object has no attribute 'dtype'

In [52]:
json_file_path = "submission.jso"

# Save dictionary to JSON file
with open(json_file_path, "w") as json_file:
    json.dump(output_data, json_file, indent=4)

: 